In [1]:
# Import libraries
import numpy as np
import pandas as pd

# Import statistical module
from scipy import stats

# Import custom modules
import acquire
import prepare
import explore

# Import visualization modules
import matplotlib.pyplot as plt
import seaborn as sns

pd.set_option('display.max_columns', None)

## Wrangle

In [2]:
# Acquire and prepare data returned as dataframe
df = acquire.get_data(query_url = False)
print(df.shape)
df = prepare.prepare_dataframe(df)
train, validate, test = prepare.split_data(df)
train.shape, validate.shape, test.shape

Found file in working directory!
(70932, 83)


((40311, 60), (6719, 60), (6719, 60))

In [3]:
train.head()

,id,name,lang,released_at,layout,highres_image,image_status,mana_cost,cmc,type_line,oracle_text,power,toughness,colors,color_identity,keywords,legalities,games,reserved,foil,nonfoil,finishes,oversized,promo,reprint,variation,set_id,set,set_name,set_type,collector_number,digital,rarity,flavor_text,artist,border_color,frame,full_art,textless,booster,story_spotlight,edhrec_rank,penny_rank,prices,promo_types,security_stamp,produced_mana,watermark,frame_effects,loyalty,printed_name,color_indicator,life_modifier,hand_modifier,content_warning,flavor_name,usd,is_foil,is_etched,is_glossy
39296,8e1e4b26-0a2b-4078-adb3-1f66315afcf5,Goblin Razerunners,en,2017-10-24,normal,True,highres_scan,{2}{R}{R},4.0,Creature — Goblin Warrior,"{1}{R}, Sacrifice a land: Put a +1/+1 counter ...",3,4,[R],[R],[],"{'standard': 'not_legal', 'future': 'not_legal...",[paper],False,False,True,[nonfoil],False,False,True,False,c77df674-0ef5-47d9-ab22-56a6e1dc901c,ddt,Duel Decks: Merfolk vs. Goblins,duel_deck,47,False,rare,Finding themselves in a new and unexplored wor...,Raymond Swanland,black,2015,False,False,False,False,10400.0,9300.0,"{'usd': '0.11', 'usd_foil': None, 'usd_etched'...",no_promo,oval,no_produced_mana,no_waterwark,no_frame_effects,0,no_printed_name,no_color_indicator,0.0,0.0,False,no_flavor,0.11,False,False,False
30783,6f2fc246-2e95-456f-aa4e-97768c4f4bb4,Rites of Initiation,en,2001-10-01,normal,True,highres_scan,{R},1.0,Instant,Discard any number of cards at random. Creatur...,0,0,[R],[R],[],"{'standard': 'not_legal', 'future': 'not_legal...","[paper, mtgo]",False,True,True,"[nonfoil, foil]",False,False,False,False,b0d90d2d-494a-4224-bfa0-36ce5ee281b1,ody,Odyssey,expansion,217,False,common,no_flavor_text,Bradley Williams,black,1997,False,False,True,False,13427.0,2777.0,"{'usd': '0.25', 'usd_foil': '3.75', 'usd_etche...",no_promo,no_security_stamp,no_produced_mana,no_waterwark,no_frame_effects,0,no_printed_name,no_color_indicator,0.0,0.0,False,no_flavor,0.25,True,False,False
26765,606206c7-1a8a-46f4-b368-cf18e02f3df8,Undying Flames,en,2005-06-03,normal,True,highres_scan,{4}{R}{R},6.0,Sorcery,Exile cards from the top of your library until...,0,0,[R],[R],[Epic],"{'standard': 'not_legal', 'future': 'not_legal...","[paper, mtgo]",False,True,True,"[nonfoil, foil]",False,False,False,False,4db16ad3-2b95-442f-bb6b-e9aa7fe7f769,sok,Saviors of Kamigawa,expansion,119,False,rare,no_flavor_text,Tsutomu Kawade,black,2003,False,False,True,False,17031.0,8830.0,"{'usd': '0.30', 'usd_foil': '2.21', 'usd_etche...",no_promo,no_security_stamp,no_produced_mana,no_waterwark,no_frame_effects,0,no_printed_name,no_color_indicator,0.0,0.0,False,no_flavor,0.30,True,False,False
8690,1f391555-4697-4492-9854-32393f473104,Bombard,en,2018-11-16,normal,True,highres_scan,{2}{R},3.0,Instant,Bombard deals 4 damage to target creature.,0,0,[R],[R],[],"{'standard': 'not_legal', 'future': 'not_legal...",[paper],False,False,True,[nonfoil],False,False,True,False,cdbedc27-4f24-4d89-ace1-8da1b36ac78e,gnt,Game Night,box,37,False,common,"""Want to sink a ship? Blow a hole in the hull....",Alex Konstad,black,2015,False,False,False,False,15067.0,7465.0,"{'usd': '0.05', 'usd_foil': None, 'usd_etched'...",no_promo,no_security_stamp,no_produced_mana,no_waterwark,no_frame_effects,0,no_printed_name,no_color_indicator,0.0,0.0,False,no_flavor,0.05,False,False,False
25570,5c1f3f52-cb9b-4b2a-bb02-6175897ae76e,Our Market Research Shows That Players Like Re...,en,2004-11-19,normal,True,highres_scan,{1}{G}{G},3.0,Creature — Elemental,Art rampage 2 (Whenever this creature becomes ...,2,2,[G],[G],[],"{'standard': 'not_legal', 'future': 'not_legal...",[paper],False,True,True,"[nonfoil, foil]",False,False,False,False,4c8bc76a-05a5-43db-aaf0-34deb347b871,unh,Unhinged,funny,107,False,common,Just call it OMRSTPLRLCNSWMTCTHTALCNEE for short.,Greg Hildebrandt,silver,2003,False,False,True,False,0.0,0.0,"{'usd': '0.20', 'usd_foil': '13.92', 'usd_etch...",no_promo,no_security_stamp,no_produced_mana,no_waterwark,no_fra

In [4]:
def add_columns_with_usd_prices_for_styles(df):
    '''
    Adds columns to dataframe with the usd prices of the card style related fields.
    '''
    # Creates a column with USD prices for cards with frame effects
    df.loc[df['frame_effects'] != 'no_frame_effects', 'frame_effects_usd'] = df['usd']
    
    # Creates a column with USD prices for cards that don't have black borders versions
    df.loc[df['border_color'] != 'black', 'border_color_usd'] = df['usd']
    
    # Creates a column with USD prices cards without flavor text
    df.loc[df['flavor_text'] != 'no_flavor_text', 'flavor_text_usd'] = df['usd']
    
    # Creates a column with USD prices non-modern style frame cards
    df.loc[df['frame'] != '2015', 'frame_usd'] = df['usd']
    
    # Creates a column with USD prices for full art cards
    df.loc[df['full_art'] == True, 'full_art_usd'] = df['usd']
    
    # Creates a column with USD prices for highres_image cards
    df.loc[df['highres_image'] == True, 'highres_images_usd'] = df['usd']
    
    # Creates a column with USD prices for non-english card
    df.loc[df['lang'] != 'en', 'lang_usd'] = df['usd']
    
    # Creates a column with USD prices for cards with non-normal layouts
    df.loc[df['layout'] != 'normal', 'layout_usd'] = df['usd']
    
    # Creates a column with USD prices for promo cards
    df.loc[df['promo_types'] != 'no_promo', 'promo_types_usd'] = df['usd']
    
    # Creates a column with USD prices for cards without a security stamp
    df.loc[df['security_stamp'] != 'no_security_stamp', 'security_stamp_usd'] = df['usd']
    
    # Creates a column with USD prices for cards without a watermark
    df.loc[df['watermark'] != 'no_waterwark', 'watermark_usd'] = df['usd']
    return df

In [5]:
add_columns_with_usd_prices_for_styles(train)
train.head(3)

/opt/homebrew/anaconda3/lib/python3.9/site-packages/pandas/core/indexing.py:1681: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = empty_value
/opt/homebrew/anaconda3/lib/python3.9/site-packages/pandas/core/indexing.py:1773: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(ilocs[0], value, pi)


,id,name,lang,released_at,layout,highres_image,image_status,mana_cost,cmc,type_line,oracle_text,power,toughness,colors,color_identity,keywords,legalities,games,reserved,foil,nonfoil,finishes,oversized,promo,reprint,variation,set_id,set,set_name,set_type,collector_number,digital,rarity,flavor_text,artist,border_color,frame,full_art,textless,booster,story_spotlight,edhrec_rank,penny_rank,prices,promo_types,security_stamp,produced_mana,watermark,frame_effects,loyalty,printed_name,color_indicator,life_modifier,hand_modifier,content_warning,flavor_name,usd,is_foil,is_etched,is_glossy,frame_effects_usd,border_color_usd,flavor_text_usd,frame_usd,full_art_usd,highres_images_usd,lang_usd,layout_usd,promo_types_usd,security_stamp_usd,watermark_usd
39296,8e1e4b26-0a2b-4078-adb3-1f66315afcf5,Goblin Razerunners,en,2017-10-24,normal,True,highres_scan,{2}{R}{R},4.0,Creature — Goblin Warrior,"{1}{R}, Sacrifice a land: Put a +1/+1 counter ...",3,4,[R],[R],[],"{'standard': 'not_legal', 'future': 'not_legal...",[paper],False,False,True,[nonfoil],False,False,True,False,c77df674-0ef5-47d9-ab22-56a6e1dc901c,ddt,Duel Decks: Merfolk vs. Goblins,duel_deck,47,False,rare,Finding themselves in a new and unexplored wor...,Raymond Swanland,black,2015,False,False,False,False,10400.0,9300.0,"{'usd': '0.11', 'usd_foil': None, 'usd_etched'...",no_promo,oval,no_produced_mana,no_waterwark,no_frame_effects,0,no_printed_name,no_color_indicator,0.0,0.0,False,no_flavor,0.11,False,False,False,NaN,NaN,0.11,NaN,NaN,0.11,NaN,NaN,NaN,0.11,NaN
30783,6f2fc246-2e95-456f-aa4e-97768c4f4bb4,Rites of Initiation,en,2001-10-01,normal,True,highres_scan,{R},1.0,Instant,Discard any number of cards at random. Creatur...,0,0,[R],[R],[],"{'standard': 'not_legal', 'future': 'not_legal...","[paper, mtgo]",False,True,True,"[nonfoil, foil]",False,False,False,False,b0d90d2d-494a-4224-bfa0-36ce5ee281b1,ody,Odyssey,expansion,217,False,common,no_flavor_text,Bradley Williams,black,1997,False,False,True,False,13427.0,2777.0,"{'usd': '0.25', 'usd_foil': '3.75', 'usd_etche...",no_promo,no_security_stamp,no_produced_mana,no_waterwark,no_frame_effects,0,no_printed_name,no_color_indicator,0.0,0.0,False,no_flavor,0.25,True,False,False,NaN,NaN,NaN,0.25,NaN,0.25,NaN,NaN,NaN,NaN,NaN
26765,606206c7-1a8a-46f4-b368-cf18e02f3df8,Undying Flames,en,2005-06-03,normal,True,highres_scan,{4}{R}{R},6.0,Sorcery,Exile cards from the top of your library until...,0,0,[R],[R],[Epic],"{'standard': 'not_legal', 'future': 'not_legal...","[paper, mtgo]",False,True,True,"[nonfoil, foil]",False,False,False,False,4db16ad3-2b95-442f-bb6b-e9aa7fe7f769,sok,Saviors of Kamigawa,expansion,119,False,rare,no_flavor_text,Tsutomu Kawade,black,2003,False,False,True,False,17031.0,8830.0,"{'usd': '0.30', 'usd_foil': '2.21', 'usd_etche...",no_promo,no_security_stamp,no_produced_mana,no_waterwark,no_frame_effects,0,no_printed_name,no_color_indicator,0.0,0.0,False,no_flavor,0.30,True,False,False,NaN,NaN,NaN,0.30,NaN,0.30,NaN,NaN,NaN,NaN,NaN


In [6]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 40311 entries, 39296 to 2198
Data columns (total 71 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   id                  40311 non-null  object        
 1   name                40311 non-null  object        
 2   lang                40311 non-null  object        
 3   released_at         40311 non-null  datetime64[ns]
 4   layout              40311 non-null  object        
 5   highres_image       40311 non-null  bool          
 6   image_status        40311 non-null  object        
 7   mana_cost           40311 non-null  object        
 8   cmc                 40311 non-null  float64       
 9   type_line           40311 non-null  object        
 10  oracle_text         40311 non-null  object        
 11  power               40311 non-null  object        
 12  toughness           40311 non-null  object        
 13  colors              40311 non-null  object 

In [7]:
avg_frame_effects = train.frame_effects_usd.mean(skipna=True)

In [8]:
avg_frame_effects = train.frame_effects_usd.mean()
avg_border_color = train.border_color_usd.mean()
avg_flavor_text = train.flavor_text_usd.mean()
avg_frame = train.frame_usd.mean()
avg_full_art = train.full_art_usd.mean()
avg_highres_images = train.highres_images_usd.mean()
avg_lang = train.lang_usd.mean()
avg_layout = train.layout_usd.mean()
avg_promo_types = train.promo_types_usd.mean()
avg_security_stamp = train.security_stamp_usd.mean()
avg_watermark = train.watermark_usd.mean()